<a href="https://colab.research.google.com/github/williamjdjulius/Big-Data-ETL/blob/main/part_one_Jewelry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-03 01:33:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.70MB/s    in 0.5s    

2023-03-03 01:33:14 (1.70 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...|2015-01-31 00:08:00|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...| 

In [6]:
# Get the number of rows in the DataFrame.
df.count()

1767753

# Transform the Data

## Create the "review_id_table".

In [11]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R135Q3VZ4DQN5N|   50423057|B00JWXFDMG|     657335467|2015-01-31 00:08:00|
|R2N0QQ6R4T7YRY|   11262325|B00W5T1H9W|      26030170|2015-01-31 00:08:00|
|R3N5JE5Y4T6W5M|   27541121|B00M2L6KFY|     697845240|2015-01-31 00:08:00|
|R2I150CX5IVY9Q|    5350721|B0006SW2WU|     569859289|2015-01-31 00:08:00|
|R1RM9ICOOA9MQ3|   24484424|B009YPDW70|     332947422|2015-01-31 00:08:00|
|R2J2KMDL10UMSH|     457767|B00QGA8XZM|     424804582|2015-01-31 00:08:00|
|R3R9ZUFA4TB4FQ|   31009358|B00L0N0E8E|     589441783|2015-01-31 00:08:00|
|R3UQ8VAQN7R6WL|    7034628|B00DMWQKWA|     517428445|2015-01-31 00:08:00|
|R1FXZ69C01JNQM|   34608636|B00THLHAGA|     320313750|2015-01-31 00:08:00|
| RY36LB4OW0FFS|   44972687|B00UXSEOHY|     589365488|2015-01-31 00:08:00|
| R6PKS54PC6ZY2|   166845

## Create the "products" Table

In [12]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LHJMHVS|FAIRY COUPLE Wome...|
|B00TPWQ6I4|Chic Alert Medica...|
|B00P837QR4|The Original Roll...|
|B00L280CN4|10pcs 18" 18 Inch...|
|B00KGU5T4Q|KONOV Mens Stainl...|
|B00JHF388K|Sterling Silver w...|
|B00FB05CES|CHOOSE YOUR COLOR...|
|B00D4KUMQ0|INBLUE Men's Stai...|
|B00GTCD2X0|Beautiful Carved ...|
|B00QUHZQP6|Authentic Chamili...|
|B00NJ317T8|Smitco LLC Silver...|
|B00658M8EC|925 Sterling Silv...|
|B00L43PV2Y|Hydia Steel Jewel...|
|B00U24CSP4|Platinum-Plated S...|
|B006OHTHZ2|316L Stainless St...|
|B00QW72KDA|PiercingJ Pair 12...|
|B006VN35L6|5mm Stainless Ste...|
|B00CGID7PY|925 Sterling Silv...|
|B00TXHSXWI|Silver Forest Kno...|
|B004Z1EQRS|PammyJ Silvertone...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [13]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   33315159|           409|
|   37638596|           305|
|   23786085|           259|
|   11799100|           178|
|   32399423|           172|
|    8156791|           163|
|    4541164|           160|
|    8241255|           153|
|   44978627|           144|
|   25468322|           139|
|    8145362|           136|
|   18581672|           124|
|   50551699|           122|
|   31581160|           121|
|    1828069|           118|
|   50245563|           108|
|   16053828|           105|
|   15245268|           105|
|   39098688|           104|
|   32405532|           103|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [14]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
from pyspark.sql.functions import col
vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
#vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|
|R2I150CX5IVY9Q|          5|            0|          0|   N|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|
|R2J2KMDL10UMSH|          5|            0|          0|   N|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|
|R1FXZ69C01JNQM|          5|            0|          0|   N|
| RY36LB4OW0FFS|          5|            0|          0|   N|
| R6PKS54PC6ZY2|          1|            4|          4|   N|
|R1LYSAH0IECVWB|          5|            0|          0|   N|
|R1UN0IZI3XQ3E2|          5|            2|          3|   N|
| RIKDXD8QGL99S|          5|            

# Load

In [15]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS


In [ ]:
# Write products_df to table in RDS


In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS
